In [2]:
"""
@author: Maziar Raissi
"""

import sys
sys.path.insert(0, '../../Utilities/')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec

np.random.seed(1234)
tf.set_random_seed(1234)

class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x, y, t, u, v, layers):
        
        X = np.concatenate([x, y, t], 1)
        
        self.lb = X.min(0)
        self.ub = X.max(0)
                
        self.X = X
        
        self.x = X[:,0:1]
        self.y = X[:,1:2]
        self.t = X[:,2:3]
        
        self.u = u
        self.v = v
        
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)        
        
        # Initialize parameters
        self.lambda_1 = tf.Variable([0.0], dtype=tf.float32)
        self.lambda_2 = tf.Variable([0.0], dtype=tf.float32)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.y_tf = tf.placeholder(tf.float32, shape=[None, self.y.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        self.v_tf = tf.placeholder(tf.float32, shape=[None, self.v.shape[1]])
        
        self.u_pred, self.v_pred, self.p_pred, self.f_u_pred, self.f_v_pred = self.net_NS(self.x_tf, self.y_tf, self.t_tf)
        
        self.loss = tf.reduce_sum(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_sum(tf.square(self.v_tf - self.v_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u_pred)) + \
                    tf.reduce_sum(tf.square(self.f_v_pred))
                    
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})        
        
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
        
    def net_NS(self, x, y, t):
        lambda_1 = self.lambda_1
        lambda_2 = self.lambda_2
        
        psi_and_p = self.neural_net(tf.concat([x,y,t], 1), self.weights, self.biases)
        psi = psi_and_p[:,0:1]
        p = psi_and_p[:,1:2]
        
        u = tf.gradients(psi, y)[0]
        v = -tf.gradients(psi, x)[0]  
        
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]
        
        v_t = tf.gradients(v, t)[0]
        v_x = tf.gradients(v, x)[0]
        v_y = tf.gradients(v, y)[0]
        v_xx = tf.gradients(v_x, x)[0]
        v_yy = tf.gradients(v_y, y)[0]
        
        p_x = tf.gradients(p, x)[0]
        p_y = tf.gradients(p, y)[0]

        f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_2*(u_xx + u_yy) 
        f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_2*(v_xx + v_yy)
        
        return u, v, p, f_u, f_v
    
    def callback(self, loss, lambda_1, lambda_2):
        print('Loss: %.3e, l1: %.3f, l2: %.5f' % (loss, lambda_1, lambda_2))
      
    def train(self, nIter): 

        tf_dict = {self.x_tf: self.x, self.y_tf: self.y, self.t_tf: self.t,
                   self.u_tf: self.u, self.v_tf: self.v}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 500 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                lambda_1_value = self.sess.run(self.lambda_1)
                lambda_2_value = self.sess.run(self.lambda_2)
                print('It: %d, Loss: %.3e, l1: %.3f, l2: %.5f, Time: %.2f' % 
                      (it, loss_value, lambda_1_value, lambda_2_value, elapsed))
                start_time = time.time()
            
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss, self.lambda_1, self.lambda_2],
                                loss_callback = self.callback)
            
    
    def predict(self, x_star, y_star, t_star):
        
        tf_dict = {self.x_tf: x_star, self.y_tf: y_star, self.t_tf: t_star}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        v_star = self.sess.run(self.v_pred, tf_dict)
        p_star = self.sess.run(self.p_pred, tf_dict)
        
        return u_star, v_star, p_star

def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

In [3]:
N_train = 5000

layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 2]

# Load Data
data = scipy.io.loadmat('Data/cylinder_nektar_wake.mat')

U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

######################################################################
######################## Noiseles Data ###############################
######################################################################
# Training Data    
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

# Training
model = PhysicsInformedNN(x_train, y_train, t_train, u_train, v_train, layers)
model.train(200000)

# Test Data
snap = np.array([100])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]

u_star = U_star[:,0,snap]
v_star = U_star[:,1,snap]
p_star = P_star[:,snap]

# Prediction
u_pred, v_pred, p_pred = model.predict(x_star, y_star, t_star)
lambda_1_value = model.sess.run(model.lambda_1)
lambda_2_value = model.sess.run(model.lambda_2)

# Error
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_p = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)

error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100

print('Error u: %e' % (error_u))    
print('Error v: %e' % (error_v))    
print('Error p: %e' % (error_p))    
print('Error l1: %.5f%%' % (error_lambda_1))                             
print('Error l2: %.5f%%' % (error_lambda_2))                  



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

It: 0, Loss: 4.252e+03, l1: -0.001, l2: 0.00100, Time: 9.76
It: 500, Loss: 4.759e+02, l1: -0.005, l2: -0.00027, Time: 62.16
It: 1000, Loss: 4.682e+02, l1: -0.011, l2: -0.00032, Time: 62.24
It: 1500, Loss: 4.659e+02, l1: -0.018, l2: -0.00054, Time: 62.18
It: 2000, Loss: 4.611e+02, l1: -0.040, l2: -0.00109, Time: 62.21
It: 2500, Loss: 4.550e+02, l1: 0.017, l2: 0.00036, Time: 62.23
It: 3000, Loss: 4.500e+02, l1: 0.068, l2: 0.00166, Time: 62.20
It: 3500, Loss: 4.405e+02, l1: 0.156, l2: 0.00501, Time: 62.23
It: 4000, Loss: 3.171e+02, l1: 0.781, l2: 0.01737, Time: 62.22
It: 4500, Loss: 1.060e+02, l1: 0.863, l2: 0.01210, Time: 62.21
It

It: 61500, Loss: 2.458e+00, l1: 0.995, l2: 0.01035, Time: 62.33
It: 62000, Loss: 2.577e+00, l1: 0.995, l2: 0.01034, Time: 62.27
It: 62500, Loss: 2.412e+00, l1: 0.995, l2: 0.01034, Time: 62.32
It: 63000, Loss: 2.391e+00, l1: 0.995, l2: 0.01034, Time: 62.32
It: 63500, Loss: 2.375e+00, l1: 0.995, l2: 0.01034, Time: 62.33
It: 64000, Loss: 2.439e+00, l1: 0.995, l2: 0.01035, Time: 62.28
It: 64500, Loss: 2.427e+00, l1: 0.995, l2: 0.01034, Time: 62.27
It: 65000, Loss: 3.003e+00, l1: 0.995, l2: 0.01038, Time: 62.26
It: 65500, Loss: 2.291e+00, l1: 0.995, l2: 0.01033, Time: 62.26
It: 66000, Loss: 6.663e+00, l1: 0.995, l2: 0.01020, Time: 62.27
It: 66500, Loss: 2.255e+00, l1: 0.995, l2: 0.01032, Time: 62.27
It: 67000, Loss: 2.250e+00, l1: 0.995, l2: 0.01033, Time: 62.27
It: 67500, Loss: 2.226e+00, l1: 0.995, l2: 0.01032, Time: 62.29
It: 68000, Loss: 2.242e+00, l1: 0.995, l2: 0.01034, Time: 62.35
It: 68500, Loss: 2.202e+00, l1: 0.995, l2: 0.01032, Time: 62.26
It: 69000, Loss: 2.581e+00, l1: 0.995, l

It: 125500, Loss: 1.201e+00, l1: 0.997, l2: 0.01034, Time: 62.27
It: 126000, Loss: 1.157e+00, l1: 0.997, l2: 0.01034, Time: 62.31
It: 126500, Loss: 1.180e+00, l1: 0.997, l2: 0.01035, Time: 62.28
It: 127000, Loss: 1.533e+00, l1: 0.997, l2: 0.01040, Time: 62.26
It: 127500, Loss: 1.929e+00, l1: 0.997, l2: 0.01037, Time: 62.32
It: 128000, Loss: 1.961e+00, l1: 0.997, l2: 0.01043, Time: 62.28
It: 128500, Loss: 1.135e+00, l1: 0.997, l2: 0.01034, Time: 62.26
It: 129000, Loss: 1.130e+00, l1: 0.997, l2: 0.01035, Time: 62.24
It: 129500, Loss: 1.143e+00, l1: 0.997, l2: 0.01034, Time: 62.28
It: 130000, Loss: 1.127e+00, l1: 0.997, l2: 0.01034, Time: 62.25
It: 130500, Loss: 1.119e+00, l1: 0.997, l2: 0.01035, Time: 62.26
It: 131000, Loss: 1.119e+00, l1: 0.997, l2: 0.01035, Time: 62.26
It: 131500, Loss: 1.131e+00, l1: 0.997, l2: 0.01035, Time: 62.26
It: 132000, Loss: 1.434e+00, l1: 0.997, l2: 0.01044, Time: 62.24
It: 132500, Loss: 1.112e+00, l1: 0.997, l2: 0.01036, Time: 62.22
It: 133000, Loss: 1.157e+

It: 189000, Loss: 8.025e-01, l1: 0.998, l2: 0.01041, Time: 62.29
It: 189500, Loss: 8.353e-01, l1: 0.998, l2: 0.01042, Time: 62.31
It: 190000, Loss: 8.133e-01, l1: 0.998, l2: 0.01042, Time: 62.28
It: 190500, Loss: 9.450e-01, l1: 0.998, l2: 0.01041, Time: 62.29
It: 191000, Loss: 8.246e-01, l1: 0.998, l2: 0.01043, Time: 62.27
It: 191500, Loss: 8.067e-01, l1: 0.998, l2: 0.01042, Time: 62.29
It: 192000, Loss: 7.945e-01, l1: 0.998, l2: 0.01042, Time: 62.27
It: 192500, Loss: 8.150e-01, l1: 0.998, l2: 0.01042, Time: 62.25
It: 193000, Loss: 8.048e-01, l1: 0.998, l2: 0.01042, Time: 62.29
It: 193500, Loss: 7.898e-01, l1: 0.998, l2: 0.01042, Time: 62.22
It: 194000, Loss: 7.893e-01, l1: 0.998, l2: 0.01043, Time: 62.30
It: 194500, Loss: 1.564e+00, l1: 0.998, l2: 0.01048, Time: 62.30
It: 195000, Loss: 1.182e+00, l1: 0.998, l2: 0.01052, Time: 62.26
It: 195500, Loss: 7.831e-01, l1: 0.998, l2: 0.01042, Time: 62.26
It: 196000, Loss: 7.935e-01, l1: 0.998, l2: 0.01043, Time: 62.29
It: 196500, Loss: 1.901e+